# Libraries imported

In [2]:
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random

# Website being scraped
### the 'base_url' will be used to join with each individual athlete link later on.

In [2]:
url = "https://olympics.com/tokyo-2020/olympic-games/en/results/all-sports/athletes.htm"

In [3]:
base_url = "https://olympics.com/tokyo-2020/olympic-games/"

# Using selenium to open chrome browser

In [37]:
driver_path = "chromedriver//chromedriver"

In [3]:
wd = webdriver.Chrome(executable_path=driver_path)

In [6]:
wd.get(url)

In [7]:
html = wd.page_source

In [8]:
soup = bs(html)

# Testing using a proxy with selenium

In [38]:
from selenium import webdriver
PROXY = "207.244.227.169:443"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome = webdriver.Chrome(options=chrome_options,executable_path=driver_path)
chrome.get("https://www.olympics.com")

# Looping through pages to grab athlete links
### I decided the best thing would be to save all the html sections needed in a list to loop through and save later.

In [12]:
links = []
for i in range(584):
    links.extend(soup.select("div.playerTag a"))
    wd.find_element_by_id('entries-table_next').click()
    time.sleep(random.uniform(5, 10))
    html = wd.page_source
    soup = bs(html)

### Needed to remove some unnecessary text from the beginning of each link text and save them.

In [ ]:
link_list = []

In [ ]:
for link in links:
    link = link["href"]
    link_list.append(link[9:])

In [ ]:
link_list

# Save/load json links
### These two functions would save and load all my links as json data to avoid repeating previous steps.

In [16]:
import json
def save_data(title,data):
    with open(title,'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [23]:
save_data('data//athlete_link_list.json', link_list)

In [31]:
import json
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [32]:
loaded_links = load_data('data//athlete_link_list.json')

In [41]:
loaded_links[3880]

'en/results/equestrian/athlete-profile-n1482494-hanna-mary.htm'

# Looping through pages and saving html
### I decided to load each link in selenium and save a portion of html from each page. Each html file is named numerically.

In [ ]:
for i in range(len(loaded_links)):
    with open(f"html_scripts//{i}.html", "w", encoding="utf-8") as file:
        wd.get(base_url + loaded_links[i])
        if i % 20 == 0 and i != 0:
            time.sleep(3)
        html = bs(wd.page_source)
        test_soup = html.select('main.wrs-content')
        file.write(str(test_soup))

# Loading pages to scrape
### Some athlete links were broken and displayed a 404 error. Added a try/catch to avoid my code from crashing from the error. 

In [1]:
import os
def load_pages():
    i = 1
    directory ='C://Users//danag//Desktop//Coding Stuff//learnpandas//html_scripts'
    for filename in os.listdir(directory):
        if filename.endswith('.html'):
            fname = os.path.join(directory,filename)
            with open(fname, 'r', encoding="utf-8") as f:
                soup = bs(f)
                try:
                    scrape_html(soup) #this calls the scraping function and passes 'soup'.
                    i = i + 1
                    print(i) #this is for me to keep track of loop iteration
                except Exception as e:
                    print(e)

# Scraping data from html
### I had to get creative selecting the exact html text from each page. The html layout of each page had very few identifying tags, which made it difficult to select certain text that I needed. This was the most time consuming section of this project.

In [98]:
# this will be the final data frame of all athletes
df = pd.DataFrame(columns=["Name", "Sex", "Age", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])

def scrape_html(soup):
    #Collecting data from html page into variables
    name = soup.find("div", class_="col-lg-9").h1.string.strip()
    noc = soup.select("div.col-9 div.playerTag")[0]['country']
    team = soup.select("div.col-9 div.playerTag a")[0].text
    age = soup.select("div.col-9 div.col-md-6")[0].text.strip('\n').split("\n\n")[1].split('\n')[1]
    sex = soup.select("div.col-9 div.col-md-6")[0].text.strip('\n').split("\n\n")[2].split('\n')[1]
    year = '2020'
    season = 'Summer'
    city = 'Tokyo'
    sport = soup.select("table.table-schedule td")[0].text.strip()
    #Storing list of all events and medals won
    event_list = []
    medal_list = []
    #looping through events in events table then appending event text to 'event_list'
    events_table = soup.find("table", class_='table-schedule').find("tbody").find_all('tr')
    for event in events_table:
        event_list.append(event.find_all('td')[1].find('a').text)
    #looping throud medals in medals table then appending medal text to 'medal_list'
    medals_table = soup.find("table", class_='table-schedule').find("tbody").find_all('tr')
    for medal in medals_table:
        if len(medal.find_all('td')[-1].find_all()) == 0: #this checks if a medal exists, if none "N/A" is added
            medal_list.append('N/A')
        else:
            medal = medal.find_all('td')[-1].find('img')['title'].split(' ')[0]
            medal_list.append(medal)
    #Creating the row of data for temporary df in a list
    athlete_info = []
    #Appending all data of athlete to 'athlete_info' list
    athlete_info.append([name, sex, age, team, noc, year, season, city, sport, event_list, medal_list])
    #Columns for temporary athlete dataframe
    columns=["Name", "Sex", "Age", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"]
    #Creating temporary dataframe with athlete data and columns listed above
    athlete_df = pd.DataFrame(athlete_info, columns = columns)
    #declaring main df as global
    global df
    #Appending temporary athlete dataframe to main dataframe
    df = df.append(athlete_df)

# Function call

In [ ]:
load_pages()

# Saving/loading original data frame
### This is the dataframe before I use the 'explode' function for the 'Event' and 'Medal' column.


In [105]:
df.to_csv('archive//unexploded_olympics.csv', index=False)

In [4]:
pd.read_csv('archive//unexploded_olympics.csv')

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,RAHAYU Apriyani,Female,23,Indonesia,INA,2020,Summer,Tokyo,Badminton,"[""Women's Doubles""]",['Gold']
1,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,"[""Women's Road Race"", ""Women's Individual Time...","['N/A', 'N/A']"
2,ABBASALI Hamideh,Female,31,Islamic Republic of Iran,IRI,2020,Summer,Tokyo,Karate,"[""Women's Kumite +61kg""]",['N/A']
3,AFONSO Salome,Female,23,Portugal,POR,2020,Summer,Tokyo,Athletics,"[""Women's 1500m""]",['N/A']
4,BERTENS Kiki,Female,29,Netherlands,NED,2020,Summer,Tokyo,Tennis,"[""Women's Singles"", ""Women's Doubles""]","['N/A', 'N/A']"
...,...,...,...,...,...,...,...,...,...,...,...
11607,TAN Kelly,Female,27,Malaysia,MAS,2020,Summer,Tokyo,Golf,"[""Women's Individual Stroke Play""]",['N/A']
11608,TAN Lianne,Female,30,Belgium,BEL,2020,Summer,Tokyo,Badminton,"[""Women's Singles""]",['N/A']
11609,TAN Qian Xiu Adele,Female,22,Singapore,SGP,2020,Summer,Tokyo,Shooting,['10m Air Rifle Women'],['N/A']
11610,TAN Sze En,Female,20,Singapore,SGP,2020,Summer,Tokyo,Artistic Gymnastics,"[""Women's Balance Beam"", ""Women's Floor Exerci...","['N/A', 'N/A']"


# Saving exploded data frame
### This is the exploded version of the data frame. It shows every event with the medal that was won for each athlete. Some athletes participated in multiple events.

In [102]:
# exploding lists in columns of "Event" and "Medal" 
final_df = df.explode(['Event', 'Medal'])

In [103]:
final_df.to_csv('archive//exploded_olympics.csv', index=False)

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,RAHAYU Apriyani,Female,23,Indonesia,INA,2020,Summer,Tokyo,Badminton,Women's Doubles,Gold
0,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,N/A
0,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,N/A
0,ABBASALI Hamideh,Female,31,Islamic Republic of Iran,IRI,2020,Summer,Tokyo,Karate,Women's Kumite +61kg,N/A
0,AFONSO Salome,Female,23,Portugal,POR,2020,Summer,Tokyo,Athletics,Women's 1500m,N/A
...,...,...,...,...,...,...,...,...,...,...,...
0,TAN Qian Xiu Adele,Female,22,Singapore,SGP,2020,Summer,Tokyo,Shooting,10m Air Rifle Women,N/A
0,TAN Sze En,Female,20,Singapore,SGP,2020,Summer,Tokyo,Artistic Gymnastics,Women's Balance Beam,N/A
0,TAN Sze En,Female,20,Singapore,SGP,2020,Summer,Tokyo,Artistic Gymnastics,Women's Floor Exercise,N/A
0,TAN Ya-Ting,Female,27,Chinese Taipei,TPE,2020,Summer,Tokyo,Archery,Women's Individual,N/A


# Testing Exploding a dataframe
### I decided to test the 'explode' function before running my scraping function. It did exactly what I needed it to do.

In [96]:
info_list = [['Daniel', 'Aguirre', "28", ['running', 'guitar', 'video games'],['1', '2', '3']]]

In [97]:
temp_df = pd.DataFrame(info_list, columns=['First Name', 'Last Name', 'Age', 'Hobbies', 'numbers'])

In [98]:
my_df = pd.DataFrame(columns=['First Name', 'Last Name', 'Age', 'Hobbies', 'numbers'])

In [99]:
temp_df

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,"[running, guitar, video games]","[1, 2, 3]"


In [100]:
my_df = my_df.append(temp_df)

In [101]:
my_df

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,"[running, guitar, video games]","[1, 2, 3]"


In [102]:
my_df.explode(['Hobbies', 'numbers'])

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,running,1
0,Daniel,Aguirre,28,guitar,2
0,Daniel,Aguirre,28,video games,3


# Cleaning data frame

In [9]:
olympics_df = pd.read_csv('archive//exploded_olympics.csv')
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,ABAD Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,ABAD Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,ABAD Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


In [10]:
#capitalized the first letter in first and last name of "Name" column
olympics_df['Name'] = olympics_df['Name'].str.title()
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Aalerud Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,Aalerud Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,Abad Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,Abad Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,Abad Nestor,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


In [11]:
#swapped the order of the names in order of first then last name
olympics_df['Name'] = pd.Series(olympics_df['Name'].str.split(' ').str[::-1].apply(lambda x : ' '.join(x)))
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Katrine Aalerud,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,Katrine Aalerud,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,Nestor Abad,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,Nestor Abad,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,Nestor Abad,Male,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


In [12]:
#reduced the "Sex" column to the first letter
olympics_df['Sex'] = olympics_df['Sex'].str[0]
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


# Final version of the data frame

In [33]:
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


In [35]:
#saving dataframe
olympics_df.to_csv('archive//olympics_2020_data.csv', index=False)